# 02. Data Cleaning Notebook

## Objectives
- Remove duplicates and handle missing values using appropriate, documented strategies.
- Standardise data formats and fix inconsistent data types.
- Produce a **clean, analysis-ready dataset** without introducing target leakage.

## Inputs
- Raw dataset from the Data Collection phase (e.g., `data/raw/lang_proficiency_results.csv` or similar).

## Outputs
- Cleaned dataset saved to `data/clean/cleaned_lang_proficiency_results.csv`.
- Summary of **before/after** data quality (row count, duplicates removed, missingness by column).
- A brief **change log** documenting cleaning decisions and transformations (e.g., imputation rules, dtype fixes).

## Additional Information
- All cleaning operations are reproducible and justified (with code + rationale).
- Backups of raw data are preserved; no edits are made to `data/raw/`.
- Outliers and missing values are handled with domain-appropriate methods and clearly documented.
- Cleaning focuses on data quality; **feature creation and leakage mitigation** are handled later in the Feature Engineering notebook.

---

# Project Directory Structure

## Change working directory

We need to change the working directory from its current folder to the folder the code of this project is currently located

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\husse\\OneDrive\\Projects\\lang-level-pred\\jupyter_notebooks'

In [2]:
from pathlib import Path

# swtich to project root directory
project_root = Path.cwd().parent
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

Working directory: c:\Users\husse\OneDrive\Projects\lang-level-pred


---

# Data loading and basic exploration
This code block imports fundamental Python libraries for data analysis and visualization and checks their versions

- pandas: For data manipulation and analysis
- numpy: For numerical computations
- matplotlib: For creating visualizations and plots

The version checks help ensure:
- Code compatibility across different environments
- Reproducibility of analysis
- Easy debugging of version-specific issues

In [4]:
# Import data analysis tools
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


print(f"pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")

pandas version: 2.3.1
NumPy version: 2.3.1
matplotlib version: 3.10.5


### List Files and Folders
- This code shows what files and folders are in our data folder and what folder we are currently in. 
- Subsequently we aren't in the right folder so the current path is set to the parent folder.

In [5]:
import os
from pathlib import Path

dataset_dir = Path("data/raw")
print(f"[INFO] Files/folders available in {dataset_dir}:")
os.listdir(dataset_dir)

[INFO] Files/folders available in data\raw:


['lang_proficiency_results_raw.csv']

## Load dataset
This code loads the dataset that is then displayed in the dataframe.

In [55]:
import pandas as pd
from pathlib import Path

# Define the path to the CSV file
file_path = Path("data/raw/lang_proficiency_results_raw.csv")

# Read the CSV file
df = pd.read_csv(file_path)

## Identifying problems in dataset

### Check Missing Values in Dataset
This code analyzes and displays missing values in our bulldozer dataset:
- Uses pandas' isna() function to identify missing values
- Counts total missing values per column using sum()
- Sorts results in descending order to highlight columns with most missing data

Previously this showed multiple missing values in some of the columns but i mistakenly ran it again after fixing the issue.

In [56]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

overall_cefr       16
listening_score    16
speaking_score     14
reading_score       9
writing_score       8
user_id             0
dtype: int64

### Check for Duplicates
This code checks if any duplicates exist in the dataset

In [57]:
# Check if there are any duplicate rows
duplicates = df.duplicated()

# Count total duplicates
print(f"Number of duplicate rows: {duplicates.sum()}")

# View the duplicate rows (if any)
df[duplicates]

Number of duplicate rows: 10


,user_id,speaking_score,reading_score,listening_score,writing_score,overall_cefr
1000,508,32.0,31.0,35.0,39.0,A1
1001,819,57.0,55.0,58.0,56.0,B1
1002,453,61.0,59.0,57.0,70.0,B1
1003,369,53.0,55.0,47.0,53.0,A2
1004,243,51.0,47.0,41.0,48.0,A2
1005,930,82.0,72.0,71.0,74.0,B2
1006,263,89.0,91.0,90.0,88.0,C1
1007,811,29.0,40.0,27.0,31.0,A1
1008,319,52.0,50.0,50.0,47.0,A2
1009,50,27.0,32.0,40.0,25.0,A1


## Check for outliers
This code checks the dataset to see if any scores exist outide of the range 100 < x < 0

In [58]:
# Check outliers for all score columns
score_columns = ['speaking_score', 'listening_score', 'reading_score', 'writing_score']

for col in score_columns:
   outliers = ((df[col] < 1) | (df[col] > 100)).sum()
   print(f"{col}: {outliers} outliers")

speaking_score: 0 outliers
listening_score: 0 outliers
reading_score: 7 outliers
writing_score: 0 outliers


In [59]:
df.dtypes

user_id              int64
speaking_score     float64
reading_score      float64
listening_score    float64
writing_score      float64
overall_cefr        object
dtype: object

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          1010 non-null   int64  
 1   speaking_score   996 non-null    float64
 2   reading_score    1001 non-null   float64
 3   listening_score  994 non-null    float64
 4   writing_score    1002 non-null   float64
 5   overall_cefr     994 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 47.5+ KB


In [32]:
# Check if any non-numeric values exist
df['listening_score'].dtype == 'object'  # True if column contains strings

False

In [33]:
# Check if any non-numeric values exist
df['speaking_score'].dtype == 'object'  # True if column contains strings

False

In [34]:
# Check if any non-numeric values exist
df['writing_score'].dtype == 'object'  # True if column contains strings

False

In [35]:
# Check if any non-numeric values exist
df['reading_score'].dtype == 'object'  # True if column contains strings

False

## Data Cleaning

### Filling Missing CEFR Levels

- Some rows in the dataset are missing their overall_cefr value.
- The CEFR level is determined by the user’s skill scores (Speaking, Listening, Reading, Writing) 
- We can infer the missing CEFRs by calculating the average score per row and mapping it to the correct level.

We use the scoring thresholds defined in this dataset:

| Average Score | CEFR |
| ------------- | ---- |
| 20–40         | A1   |
| 41–55         | A2   |
| 56–70         | B1   |
| 71–85         | B2   |
| 86–92         | C1   |
| 93+           | C2   |


Example:

- If a user has scores (Speaking=72, Reading=74, Listening=70, Writing=71), the row average = 71.75 → this falls in the 71–85 range, so their overall_cefr is assigned as B2.

In [61]:
# Function to map average score → CEFR level
def assign_cefr(score):
    if score <= 40: return "A1"
    elif score <= 55: return "A2"
    elif score <= 70: return "B1"
    elif score <= 85: return "B2"
    elif score <= 92: return "C1"
    else: return "C2"

# Identify rows with missing CEFR
mask = df["overall_cefr"].isna()

# Compute row averages for those rows (ignoring NaNs in skills)
row_means = df.loc[mask, ["speaking_score", "reading_score", "listening_score", "writing_score"]].mean(axis=1, skipna=True)

# Assign CEFR levels
df.loc[mask, "overall_cefr"] = row_means.apply(assign_cefr)

#### Verify fix
- This code verifies there are no remaining mising values in the overall_cefr column.

In [62]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

listening_score    16
speaking_score     14
reading_score       9
writing_score       8
user_id             0
overall_cefr        0
dtype: int64

### Handling Missing Values by Group Mean Imputation

In this dataset, each user’s skill scores (Speaking, Listening, Reading, Writing) directly influence their overall CEFR level.
If we replaced missing values with a global mean across all CEFR levels, it would distort the relationship between individual skills and the CEFR classification.

Instead, we fill missing values using the mean score within the same CEFR group.
This way, the imputation reflects the performance range typical of each level.

Example:

- If a speaking_score is missing for a B2 user:
- Collect all B2 users’ speaking scores.
- Compute the mean of those scores.
- Fill the missing value with this mean.
- This preserves the internal consistency of the dataset while ensuring no missing values remain.


In [63]:
# Fill missing values by CEFR group mean
for col in ["speaking_score", "reading_score", "listening_score", "writing_score"]:
    df[col] = df.groupby("overall_cefr")[col].transform(lambda x: x.fillna(x.mean()))

### Verify fix
- This code verifies the missing values where successfully addressed and should show no remaining missing values in any cell.

In [64]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

user_id            0
speaking_score     0
reading_score      0
listening_score    0
writing_score      0
overall_cefr       0
dtype: int64

## Fixing Inconsistent Categorical Labels

In the dataset, some overall_cefr values appear in lowercase (e.g., "b2", "c2") while most are uppercase ("B2", "C2").
This inconsistency could cause issues when grouping or filtering by CEFR levels.

To standardize the labels, we convert all values in the overall_cefr column to uppercase:


In [65]:
# Capitalize all CEFR values to ensure consistency
df['overall_cefr'] = df['overall_cefr'].str.upper()

## Round Scores to Integers
- Since all scores are percentages, they should be stored as whole numbers.
- We round any decimal values to the nearest integer and cast the columns to integer type.
- This maintains consistency in the dataset and ensures that all scores are expressed as whole percentage values between 0 and 100.

In [66]:
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']] = \
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']].round(0).astype(int)

### Verify fix 
- This code gives a brief look at the updated dataset with the now rounded values

In [67]:
df.head(60)

,user_id,speaking_score,reading_score,listening_score,writing_score,overall_cefr
0,1,26,40,28,33,A1
1,2,91,92,89,87,C1
2,3,61,66,66,57,B1
3,4,65,60,55,55,B1
4,5,77,76,83,78,B2
5,6,58,66,61,56,B1
6,7,50,46,55,52,A2
7,8,78,71,82,81,B2
8,9,45,51,42,49,A2
9,10,64,64,68,56,B1


## Handling Outliers and Missing Values

In this dataset, each test score should be between 0 and 100.
However:

- Some outliers may exist due to data entry errors (e.g., -5 or 105).

As a result we address this issue in 2 steps:

- 1: Outlier Detection & Replacement
    - Any value < 0 or > 100 is considered invalid.
    - Such values are replaced with NaN so they can be treated consistently with other missing values.

- 2: Missing Value Imputation by CEFR Group Mean
    - Missing values (those just created from outlier replacement) are them imputed with the mean score of the same CEFR group.
    - This preserves the relationship between CEFR level and test scores.

In [68]:
# Define score columns
score_cols = ['speaking_score', 'reading_score', 'listening_score', 'writing_score']

# 1. Replace outliers (<0 or >100) with NaN
for col in score_cols:
    df.loc[(df[col] < 0) | (df[col] > 100), col] = pd.NA

# 2. Fill missing values with group mean (based on overall_cefr)
for col in score_cols:
    df[col] = df.groupby('overall_cefr')[col].transform(lambda x: x.fillna(x.mean()))

### Verify fix

The code below counts outliers per skill column (speaking_score, reading_score, listening_score, writing_score).

This helps us:

- Confirm whether outliers still exist.
- Decide if replacement or imputation is still needed.


In [69]:
# Define score columns
score_cols = ['speaking_score', 'reading_score', 'listening_score', 'writing_score']

# Count outliers in each column (values <0 or >100)
outliers_count = {}
for col in score_cols:
    outliers_count[col] = ((df[col] < 0) | (df[col] > 100)).sum()

print("Number of outliers per column:")
print(outliers_count)

Number of outliers per column:
{'speaking_score': np.int64(0), 'reading_score': np.int64(0), 'listening_score': np.int64(0), 'writing_score': np.int64(0)}


## Dropping Unnecessary Columns

In some cases, certain columns in the dataset may not be relevant for analysis or modeling.  
We can remove such columns to simplify the dataset and reduce noise.

**Example:**  
The `user_id` column is an identifier and does not contribute to predictive modeling, so it can be dropped.

In [70]:
# Drop the 'user_id' column
df.drop('user_id', axis=1, inplace=True)

## Removing Duplicate Rows

Duplicate rows can occur due to data entry errors or merging issues.
If left unhandled, they can bias analysis (e.g., counting a learner’s scores twice).

Approach:

- Identify duplicate rows using df.duplicated().
- For each duplicate row, add a small random adjustment (e.g., ±1 to ±3) to numeric scores (speaking_score, reading_score, listening_score, writing_score).
- Ensure the adjusted scores still stay within the valid range 0–100.
- Keep the CEFR level the same (since tiny score changes wouldn’t realistically change overall level).
- Remove them with drop_duplicates().
- Verify that no duplicates remain.


In [73]:
# 1. Check number of duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
df.duplicated().sum() 


Number of duplicate rows: 292


np.int64(292)

In [74]:
# Find duplicates
duplicates_mask = df.duplicated(keep=False)
duplicates = df[duplicates_mask]

# Apply small random adjustments to duplicates
for col in ['speaking_score', 'reading_score', 'listening_score', 'writing_score']:
    df.loc[duplicates.index, col] = (
        df.loc[duplicates.index, col] + np.random.choice([-2, -1, 1, 2], size=len(duplicates))
    ).clip(0, 100)  # keep scores within 0-100

In [76]:
# 1. Check number of duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
df.duplicated().sum()

# 2. Remove remaining duplicates
df = df.drop_duplicates()

Number of duplicate rows: 4


## Recalculate CEFR Levels

After addressing missing values, outliers, and duplicates, we recalculate the overall CEFR levels to ensure they align with the updated skill scores.

- We calculate each user’s average score across speaking, reading, listening, and writing.
- We then assign a CEFR level using the following scale:


| Average Score | CEFR |
| ------------- | ---- |
| 20–40         | A1   |
| 41–55         | A2   |
| 56–70         | B1   |
| 71–85         | B2   |
| 86–92         | C1   |
| 93+           | C2   |


In [96]:
# Step: Recalculate CEFR based on average scores

def assign_cefr(avg):
    if 20 <= avg <= 40:
        return "A1"
    elif 40 < avg <= 55:   # continuous, no gap
        return "A2"
    elif 55 < avg <= 70:
        return "B1"
    elif 70 < avg <= 85:
        return "B2"
    elif 85 < avg <= 92:
        return "C1"
    elif avg > 92:
        return "C2"
    else:
        return None

# Compute average score
df["average_score"] = df[["speaking_score", "reading_score", "listening_score", "writing_score"]].mean(axis=1)

# Assign new CEFR based on average score
df["overall_cefr"] = df["average_score"].apply(assign_cefr)

# Drop the helper column
df.drop(columns=["average_score"], inplace=True)

## Round Scores to Integers
- Since all scores are percentages, they should be stored as whole numbers.
- We round any decimal values to the nearest integer and cast the columns to integer type.
- This maintains consistency in the dataset and ensures that all scores are expressed as whole percentage values between 0 and 100.

In [107]:
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']] = \
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']].round(0).astype(int)

## Summary Statistics Check

Let's take a look at the summary statistics for each score column in our cleaned dataset. This helps us understand the distribution, identify any remaining anomalies, and get a sense of the central tendency and spread of the data.


In [112]:
df.describe()

,speaking_score,reading_score,listening_score,writing_score
count,1006.000000,1006.000000,1006.000000,1006.000000
mean,61.152087,61.156064,61.203777,60.982107
std,21.256026,21.456621,21.425777,21.227966
min,23.000000,23.000000,23.000000,23.000000
25%,43.000000,43.000000,42.000000,43.000000
50%,60.500000,61.000000,60.000000,60.000000
75%,78.000000,79.000000,80.000000,79.750000
max,100.000000,100.000000,100.000000,100.000000


### Correct data type check

The code below displays info about the dataset, specifically checking the data types of each column:

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1004 entries, 0 to 1009
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   speaking_score   1004 non-null   int64 
 1   reading_score    1004 non-null   int64 
 2   listening_score  1004 non-null   int64 
 3   writing_score    1004 non-null   int64 
 4   overall_cefr     1004 non-null   object
dtypes: int64(4), object(1)
memory usage: 47.1+ KB


## Saving the Cleaned Dataset

After completing the data cleaning process, it's good practice to save the cleaned dataset for future use. This ensures that we don't have to repeat the cleaning steps and keeps our project organized.

We can save the dataset to a specific folder, for example `data/clean/`, or simply in the current working directory. Using a folder structure helps separate raw and cleaned data, making the workflow more reproducible.

In [117]:
from pathlib import Path

file_path = Path("data/clean/cleaned_lang_proficiency_results.csv")
file_path.parent.mkdir(parents=True, exist_ok=True)  # Creates folder if it doesn't exist
df.to_csv(file_path, index=False)

## Conclusion

In this notebook, we performed a thorough cleaning of our dataset. Key steps included:

- Handling missing values by either filling or correcting them.
- Fixing outliers and erroneous values in the numeric score columns.
- Ensuring consistency across all columns, particularly the `overall_cefr` levels.
- Dropping unnecessary or redundant columns.
- Adjusting duplicate values appropriately.
- Verifying that all data types were appropriate for analysis.

After cleaning, the dataset is now consistent, complete, and ready for further analysis or modeling. Summary statistics confirm that all scores are within reasonable ranges, and categorical labels accurately reflect the underlying numeric data. These final checks give us confidence in the integrity of the dataset moving forward.

### Next steps
- 03_eda.ipynb: This notebook will focus on exploring the cleaned dataset. We will generate visualizations, examine distributions, check correlations, and identify patterns or trends in the data to inform future modeling decisions.